In [ ]:
import pickle
import pandas as pd
import concurrent.futures
import pathlib

## Uniqueify by taking max both ways

### Experiment

#### Load and see how many candidates there are

In [ ]:
matches = pd.read_csv("m-TM.csv", delimiter="|")

In [ ]:
len(matches)

In [ ]:
# for computing upper bound on number of extracted matches
g1 = matches.groupby(by=["a_FT", "a_Kipnr", "a_Løbenr"], as_index=False)
g2 = matches.groupby(by=["b_FT", "b_Kipnr", "b_Løbenr"])
l1=list(g1.size().groupby(level=0).size().iteritems())
l2=list(g2.size().groupby(level=0).size().iteritems())
print(l1)
print(l2)

In [ ]:
g1 = matches.groupby(by=["a_FT", "a_Kipnr", "a_Løbenr"])
idx = g1["p"].transform(max) == matches["p"]
matches = matches[idx]
print(len(matches))

In [ ]:
g2 = matches.groupby(by=["b_FT", "b_Kipnr", "b_Løbenr"])
idx = g2["p"].transform(max) == matches["p"]
matches = matches[idx]
print(len(matches))

In [ ]:
len(matches)

In [ ]:
for (a_year, a_count), (b_year, b_count) in zip(l1, l2):
    print(a_year, b_year, min(a_count, b_count))
print(sum(min(a, b) for ((_, a), (_, b)) in zip(l1,l2)))

In [ ]:
# so how many left?
g1 = matches.groupby(by=["a_FT", "a_Kipnr", "a_Løbenr"], as_index=False)
g2 = matches.groupby(by=["b_FT", "b_Kipnr", "b_Løbenr"])
l1=list(g1.size().groupby(level=0).size().iteritems())
l2=list(g2.size().groupby(level=0).size().iteritems())

In [ ]:
for (a_year, a_count), (b_year, b_count) in zip(l1, l2):
    print(a_year, b_year, min(a_count, b_count))
print(sum(min(a, b) for ((_, a), (_, b)) in zip(l1,l2)))

#### There are probably still duplicates
Obvious if `len(matches)` above is greater than the sum in the cell directly above.

In [ ]:
indexed = matches.set_index(["a_FT", "a_Kipnr", "a_Løbenr"])

In [ ]:
d = indexed.index.get_duplicates()

In [ ]:
indexed.loc[d]

In [ ]:
len(matches)

In [ ]:
matches.drop_duplicates(subset=["a_FT", "a_Løbenr", "a_Kipnr"], inplace=True)
matches.drop_duplicates(subset=["b_FT", "b_Løbenr", "b_Kipnr"], inplace=True)

In [ ]:
len(matches)

In [ ]:
matches

### Apply

In [ ]:
def uniqueify_max(matches):
    # take max-probability b for each a
    g = matches.groupby(by=["a_FT", "a_Kipnr", "a_Løbenr"])
    idx = g["p"].transform(max) == matches["p"]
    matches = matches[idx]
    del(g)
    del(idx)
    # and then max-probability a for each b
    g = matches.groupby(by=["b_FT", "b_Kipnr", "b_Løbenr"])
    idx = g["p"].transform(max) == matches["p"]
    matches = matches[idx]
    del(g)
    del(idx)
    # drop any duplicate a or b
    matches.drop_duplicates(subset=["a_FT", "a_Løbenr", "a_Kipnr"], inplace=True)
    matches.drop_duplicates(subset=["b_FT", "b_Løbenr", "b_Kipnr"], inplace=True)
    return matches

In [ ]:
def process(fn):
    matches = pd.read_csv(str(fn), delimiter="|")
    matches = matches[matches.p > 0.8]
    matches = uniqueify_max(matches)
    return matches

In [ ]:
with concurrent.futures.ProcessPoolExecutor(max_workers=10) as tpe:
    res = tpe.map(process, pathlib.Path("nn-match").glob("*.csv"))

In [ ]:
res = list(res)

In [ ]:
len(res)

In [ ]:
matches = pd.concat(res)

In [ ]:
del(res)

In [ ]:
len(matches)

In [ ]:
# so how many left?
g1 = matches.groupby(by=["a_FT", "a_Kipnr", "a_Løbenr"], as_index=False)
g2 = matches.groupby(by=["b_FT", "b_Kipnr", "b_Løbenr"])
l1=list(g1.size().groupby(level=0).size().iteritems())
l2=list(g2.size().groupby(level=0).size().iteritems())

In [ ]:
for (a_year, a_count), (b_year, b_count) in zip(l1, l2):
    print(a_year, b_year, min(a_count, b_count))
print(sum(min(a, b) for ((_, a), (_, b)) in zip(l1,l2)))

In [ ]:
g = matches.groupby("a_FT")

In [ ]:
s = 0
for a_year, size in g.size().iteritems():
    print(a_year, size)
    s += size
s

In [ ]:
df = pd.read_pickle("../scripts/dataframe.pickled")

In [ ]:
g2 = df.groupby("FT")

In [ ]:
l = list(g2.size().iteritems())

In [ ]:
for (a_year, a_size), (b_year, b_size), (our_year, our_size) in zip(l, l[1:], g.size().iteritems()):
    ul = min(a_size, b_size)
    print(a_year, "->", b_year, our_size, "/", ul, our_size/ul)

In [ ]:
hmm = matches.p.describe()

In [ ]:
print(hmm.to_latex())

## There are probably still duplicates
Obvious if `len(matches)` above is greater than the sum in the cell directly above.

In [ ]:
indexed = matches.set_index(["a_FT", "a_Kipnr", "a_Løbenr"])

In [ ]:
d = indexed.index.get_duplicates()

In [ ]:
indexed.loc[d]

In [ ]:
len(matches)

In [ ]:
matches.drop_duplicates(subset=["a_FT", "a_Løbenr", "a_Kipnr"], inplace=True)
matches.drop_duplicates(subset=["b_FT", "b_Løbenr", "b_Kipnr"], inplace=True)

In [ ]:
len(matches)

In [ ]:
matches

# Now only sensible matches left
So let's look at these

## Inspect some matches

In [ ]:
df = pd.read_pickle("../scripts/indexed.pickled")

In [ ]:
lookup = {}
for t in df.itertuples():
    lookup[t.Index] = t

In [ ]:
del(df)

In [ ]:
header = "_|Amt|Herred|Sogn|Navn|Køn|Fødested|Fødeår|Civilstand|Position|Erhverv|Fornavn|Kipnr|Løbenr|Group".split("|")

In [ ]:
matches.sort_values(by="p", inplace=True, ascending=False)

In [ ]:
ss = matches[abs(matches.p - 0.935037) < 0.0001]

In [ ]:
import textwrap

In [ ]:
sh = textwrap.shorten

In [ ]:
count = 0
for _, a_FT, a_Kip, a_Løb, b_FT, b_Kip, b_Løb, prob in matches.itertuples():
    count += 1
    a_key = (a_FT, a_Kip, a_Løb)
    b_key = (b_FT, b_Kip, b_Løb)
    a = lookup[a_key]
    b = lookup[b_key]
    print("\\multicolumn{3}{l}{" + "Match from " + str(a_FT) + " to " + str(b_FT) + "}\\\\")
    print("\\multicolumn{3}{l}{" + "Assigned probability: " + str(prob) + "}\\\\")
    #print("Match from", a_FT, "to", b_FT)
    #print("Assigned probability:", prob)
    for i in range(1, 11):
        #s = "{:10s} {:30s} {:30s}".format(header[i], str(a[i]), str(b[i]))
        s = "{}&{}&{}\\\\".format(header[i], sh(str(a[i]), 40), sh(str(b[i]), 40))
        if a[i] != b[i]:
            s = '\033[95m' + s + "\033[0m"
        print(s)
    print("\midrule")
    if count == 5:
        break

In [ ]:
iiii